# Building a state of the art bacterial classifier with fastai

> ***This is a modified version of the notebook [Bacteria Classification with fast.ai](https://ml-showcase.paperspace.com/projects/bacteria-classification-fast-ai-dibas) created by The Gradient Team at Paperspace***. 

> The code has been slightly updated to support the most recent fastai version and some additional content added. 

The fast.ai library has been breaking records as students use it to produce state of the art results on a myriad of tasks. Some examples given in the course include environmental sound classification and handwritten devanagari prediction.

In this notebook we'll use the library for state of the art bacteria classification with the [DIBaS (Digital Image of Bacterial Species) dataset](http://misztal.edu.pl/software/databases/dibas/). See also _Zieliński, Bartosz, et al. ["Deep learning approach to bacterial colony classification."](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0184554) PloS one 12.9 (2017)_ paper link]. DIBaS contains 660 images, with 33 different genera and species of bacteria.

You can also check out the full blog post on [Building a Bacterial Classifier with fast.ai](https://blog.paperspace.com/building-a-state-of-the-art-bacterial-classifier-with-paperspace-gradient-and-fast-ai/) by Harsh Sikka.

> Note: Running this notebook outside Gradient requires some modifications.

## Install and load libraries

In [ ]:
!pip install -Uqq fastbook
!pip install bs4

In [ ]:
import os
import requests
import shutil
import zipfile
import matplotlib.pyplot as plt

import fastbook
fastbook.setup_book()
from fastbook import *

from fastai.vision.all import *
from bs4 import BeautifulSoup

# Download and load bacteria dataset

In [ ]:
if not os.path.exists('/storage/dibas_zips'):
    os.makedirs('/storage/dibas_zips')

if not os.path.exists('/storage/dibas_images'):
    os.makedirs('/storage/dibas_images')

In [ ]:
if not os.path.exists('/storage/dibas_zips/Acinetobacter.baumanii.zip'):
    # Parse the webpage; images are saved in a separate .zip file for each strain of bacteria
    url = 'http://misztal.edu.pl/software/databases/dibas/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    links = [tag['href'] for tag in soup.findAll('a')]

    for link in links:
        if ".zip" in link:
            file_name = link.partition("/dibas/")[2]
            urllib.request.urlretrieve(link, '/storage/dibas_zips/' + file_name) 
            zip_ref = zipfile.ZipFile('/storage/dibas_zips/' + file_name, 'r')
            zip_ref.extractall('/storage/dibas_images/')   
            zip_ref.close()
            print("Downloaded and extracted: " + file_name)

In [ ]:
path = Path('/storage/dibas_images/')
path.ls()

### Check the images and delete the ones that won't load

In [ ]:
failed = verify_images(path.ls(file_exts='.tif'))

In [ ]:
failed

Delete the ones that didn't load:

In [ ]:
failed.map(Path.unlink)

# Create dataloaders

In [ ]:
def label_func(fn):
    return str(fn.name).split('_')[0]

In [ ]:
item_tfms = Resize(400, method='pad')

In [ ]:
batch_tfms = [Resize(300, method='pad'), Flip(), Zoom(),
              Contrast(), Rotate(30), Normalize.from_stats(*imagenet_stats)]

In [ ]:
db = DataBlock(blocks=[ImageBlock(), CategoryBlock()], 
                get_items= partial(get_image_files, recurse=False),
                get_y = label_func,
                splitter=RandomSplitter(valid_pct=0.3),
                item_tfms=item_tfms,
                batch_tfms=batch_tfms
              )

In [ ]:
dls = db.dataloaders(path, bs=64)

In [ ]:
dls.show_batch(figsize=(16,16))

# Train models

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=[accuracy, error_rate])

Find a good learning rate:

In [ ]:
lr_min, lr_steep = learn.lr_find(suggestions=True)

In [ ]:
lr_min, lr_steep

In [ ]:
learn.fine_tune(10, base_lr=lr_min/10)

# Evaluate results

In [ ]:
learn.show_results(figsize=(16,16))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(4, figsize=(16,12))

In [ ]:
interp.plot_confusion_matrix(figsize=(16,16))

In [ ]:
0.015000 * len(learn.dls.valid_ds.items)

Only three errors among the 200 images in the validation set.

# Your turn!

1. To compare with the results of the original paper one should design a more careful validatation setup. In the paper they did a 25 fold cross-validation. Try to do something similar (but with a smaller number of folds if you don't have time for 25 training runs).
2. Try other CNN network architectures.
3. Look into other data augmentation strategies.
4. Add class activation maps (CAM and/or gradCAM) to investigate what features the model used to make its classifications.